In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV

In [4]:
osteo = pd.read_csv('osteoporosis.csv')

In [5]:
osteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1958 entries, 0 to 1957
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Id                   1958 non-null   int64 
 1   Age                  1958 non-null   int64 
 2   Gender               1958 non-null   object
 3   Hormonal Changes     1958 non-null   object
 4   Family History       1958 non-null   object
 5   Race/Ethnicity       1958 non-null   object
 6   Body Weight          1958 non-null   object
 7   Calcium Intake       1958 non-null   object
 8   Vitamin D Intake     1958 non-null   object
 9   Physical Activity    1958 non-null   object
 10  Smoking              1958 non-null   object
 11  Alcohol Consumption  970 non-null    object
 12  Medical Conditions   1311 non-null   object
 13  Medications          973 non-null    object
 14  Prior Fractures      1958 non-null   object
 15  Osteoporosis         1958 non-null   int64 
dtypes: int

In [15]:
#view the objects to see what objects are present in them and if any NaN are present in them 
for col in osteo.columns:
    print(osteo[col])
#not sure if we want to change the labels to numerical values to make the data easier to handle, if not thats okay too, we can use algorithms to count!

0       1734616
1       1419098
2       1797916
3       1805337
4       1351334
         ...   
1953    1991635
1954    1528601
1955    1990957
1956    1779848
1957    1282963
Name: Id, Length: 1958, dtype: int64
0       69
1       32
2       89
3       78
4       38
        ..
1953    19
1954    23
1955    34
1956    25
1957    26
Name: Age, Length: 1958, dtype: int64
0       Female
1       Female
2       Female
3       Female
4         Male
         ...  
1953    Female
1954    Female
1955    Female
1956      Male
1957    Female
Name: Gender, Length: 1958, dtype: object
0               Normal
1               Normal
2       Postmenopausal
3               Normal
4       Postmenopausal
             ...      
1953            Normal
1954    Postmenopausal
1955    Postmenopausal
1956    Postmenopausal
1957    Postmenopausal
Name: Hormonal Changes, Length: 1958, dtype: object
0       Yes
1       Yes
2        No
3        No
4       Yes
       ... 
1953    Yes
1954    Yes
1955     No
1956    